In [ ]:
!pip install tflearn

In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
# This Python 3 environment comes with many helpful analytiimport tflearn
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import highway_conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization, batch_normalization
from tflearn.layers.estimator import regression
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
from tflearn.layers.conv import conv_2d, max_pool_2d

from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import pandas as pd
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import skimage
from skimage import io

In [ ]:
path='/kaggle/input/nn23-sports-image-classification/Train'
path_directory_list = os.listdir(path)
lables=[]
images=[]
image_path=[]

In [ ]:
for dir in path_directory_list:
        part = dir.split('_')[0]
        lables.append(part)
        image_path.append(path+'/'+dir)
    
   
   
   
       

In [ ]:
df=pd.DataFrame()
df['image']=image_path
df['Lable']=lables
df['Lable'].unique()

In [ ]:
sns.countplot(df['Lable'])

In [ ]:
class_1,class_2,class_3,class_4,class_5,class_6 = df['Lable'].value_counts()
c1 = df[df['Lable'] == 'Yoga']
c2 = df[df['Lable'] == 'Football']
c3 = df[df['Lable'] == 'Swimming']
c4 = df[df['Lable'] == 'Tennis']
c5 = df[df['Lable'] == 'Rowing']
c6 = df[df['Lable'] == 'Basketball']

df_1 = c2.sample(class_1, replace=True)
df_2 = c3.sample(class_1, replace=True)
df_3 = c4.sample(class_1, replace=True)
df_4 = c5.sample(class_1, replace=True)
df_5 = c6.sample(class_1, replace=True)

oversampled_df = pd.concat([c1,df_1,df_2,df_3,df_4,df_5], axis=0)
oversampled_df = oversampled_df.sort_index(ascending=True)
oversampled_df=oversampled_df.reset_index(drop=True)
oversampled_df.head()

In [ ]:
sns.countplot(oversampled_df['Lable'])

In [ ]:
oversampled_df['Lable'].value_counts()

In [ ]:
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])

In [ ]:
lables.clear()
for i in range(len(oversampled_df)):
        lables.append(oversampled_df['Lable'][i])
        img_data = cv2.imread(str(oversampled_df['image'][i]))
        img_data = cv2.resize(img_data, (128, 128))
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        #img_data = skimage.color.rgb2gray(img_data)
        img_data = cv2.filter2D(src=img_data, ddepth=-1, kernel=kernel)
        img_data = cv2.fastNlMeansDenoising(img_data, None, 20, 7, 21) 
        images.append(np.array(img_data))


In [ ]:
print(images[0].shape)

In [ ]:
plt.imshow(images[0])

In [ ]:
lables=np.array(lables)
lables=lables.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(lables)
y = y.toarray()

In [ ]:
X=np.array(images)
X=(X - np.min(X)) / (np.max(X) - np.min(X))
#X=X.reshape(-1,128,128,1)



# **Data augmantation**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2 ,random_state=700, shuffle=True)
generator= ImageDataGenerator(
        featurewise_center =True,
        featurewise_std_normalization = True,
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


generator.fit(x_train)


# **Modeling**

In [ ]:
#optimizer and learning rate
opt = Adam(lr=0.0001)

In [ ]:
cnn4 = Sequential()
cnn4.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(128,128,3)))
cnn4.add(MaxPool2D())

cnn4.add(Conv2D(32, 3, padding="same", activation="relu"))
cnn4.add(MaxPool2D())

cnn4.add(Conv2D(64, 3, padding="same", activation="relu"))
cnn4.add(MaxPool2D())


cnn4.add(Conv2D(64, 3, padding="same", activation="relu"))
cnn4.add(MaxPool2D())



cnn4.add(Dropout(0.4))



cnn4.add(Flatten())
cnn4.add(Dense(128,activation="relu"))
cnn4.add(Dense(6, activation="softmax"))

cnn4.summary()
cnn4.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
cnn4.fit(
  x_train,
  y_train,
  epochs=250,
  validation_data=(x_test,y_test),
   batch_size=96
)

In [ ]:
model = Sequential(name='VGG')
model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))
model.add(Dense(units=6, activation="softmax"))
model.summary()
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(
  x_train,
  y_train,
  epochs=50,
  validation_data=(x_test,y_test),
  batch_size=96
)

In [ ]:
network = input_data(shape=[None, 128, 128, 3], name='input')

#highway convolutions with pooling and dropout

network = highway_conv_2d(network, 16, 5, activation='relu')
network = highway_conv_2d(network, 16, 5, activation='relu')
network = highway_conv_2d(network, 16, 5, activation='relu')
network = max_pool_2d(network, 5)
network = batch_normalization(network)
network = fully_connected(network, 128, activation='relu')
network = fully_connected(network, 256, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 6, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.0001,loss='categorical_crossentropy', name='target')

# Training
model2 = tflearn.DNN(network, tensorboard_dir='log', tensorboard_verbose=3)
model2.fit(x_train, y_train, n_epoch=100, validation_set=(x_test, y_test),show_metric=True,batch_size=96,run_id='convnet_highway_mnist')

In [ ]:
ops.reset_default_graph()
conv_input = input_data(shape=[None, 128, 128,3], name='input')
conv1 = conv_2d(conv_input, 32, 3, activation='relu')
pool1 = max_pool_2d(conv1, 3)

conv2 = conv_2d(pool1, 64, 3, activation='relu')
pool2 = max_pool_2d(conv2, 3)

fully_layer = fully_connected(pool2, 1024, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy', name='targets')
model3 = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)

In [ ]:
model3.fit({'input': x_train}, {'targets': y_train}, n_epoch=50,
          validation_set=({'input': x_test}, {'targets': y_test}),
          snapshot_step=500, show_metric=True,batch_size=96, run_id='CNN')

In [ ]:
network = input_data(shape=[None, 128, 128, 3] , name='input')
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 3)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 3)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 6, activation='softmax')
network = regression(network, optimizer='adam',loss='categorical_crossentropy',learning_rate=0.0001)

# Train using classifier
model4 = tflearn.DNN(network, tensorboard_verbose=0)
model4.fit(x_train, y_train, n_epoch=200, shuffle=True, validation_set=(x_test, y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

In [ ]:
test_path='/kaggle/input/nn23-sports-image-classification/Test'
test_path_directory_list = os.listdir(test_path)
test_images=[]
test_images_path=[]

In [ ]:
for dir in test_path_directory_list:
        img_data = cv2.imread(test_path+'/'+dir)
        img_data = cv2.resize(img_data, (128, 128))
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        #img_data = skimage.color.rgb2gray(img_data)
        img_data = cv2.filter2D(src=img_data, ddepth=-1, kernel=kernel)
        img_data = cv2.fastNlMeansDenoising(img_data, None, 20, 7, 21) 
        test_images.append(np.array(img_data))
        test_images_path.append(dir)
 
       


In [ ]:
test_images=np.array(test_images)
test_images=(test_images - np.min(test_images)) / (np.max(test_images) - np.min(test_images))
#test_images=test_images.reshape(-1,128,128,1)


In [ ]:
from IPython.core.display import HTML
import pandas as pd

def predections_csv(predictions):
    finalPrediction = []
    for i in predictions:
    
        finalPrediction.append(pd.Series(i).idxmax())
        
  
   
    data = {'image_name':test_images_path , 'label':finalPrediction}
    df = pd.DataFrame(data)
    df.to_csv('df.csv',index=False)
    html = '<a href=df.csv>Download csv</a>'
    html = html.format(title='Download csv',filename='df.csv')
    return HTML(html)

In [ ]:
predictions=model3.predict(test_images)
predections_csv(predictions)            
        
        

In [ ]:
predictions=model.predict(test_images)
predections_csv(predictions)    